In [1]:
import csv
import pandas as pd
import json
import pickle

def preprocess(ids, texts, indexes):
    reasult = {'id': [], 'text': [], 'class': []}
    if indexes != 'n/a':
        is_hate = json.loads(indexes)
    else:
        is_hate = []
    post_ids = [post_id_line[2:].strip() for post_id_line in ids.strip().split('\n')]
    posts = [text[2:].strip() for text in texts.strip().split('\n')]
    for index, row in enumerate(zip(post_ids, posts)):
        if len(row[1]) > 0:
            if row[0][0] == '.':
                new_post_id = row[0][1:].strip()
            else:
                new_post_id = row[0].strip()
            reasult['id'].append(new_post_id)
            if row[1][0] == '.':
                stripped_text = row[1][1:].strip()
            else:
                stripped_text = row[1].strip()
            reasult['text'].append(stripped_text)
            if index in is_hate:
                reasult['class'].append(1)
            else:
                reasult['class'].append(2)
    return pd.DataFrame.from_dict(reasult)


with open('hsd/Reddit/gab.csv') as csv_file:
    posts_df = pd.DataFrame(columns = ['id', 'text', 'class']) 
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader, None)
    for row in csv_reader:
        posts_df = posts_df.append(preprocess(row[0], row[1], row[2]), ignore_index=True)

with open('hsd/Reddit/reddit.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader, None)
    for row in csv_reader:
        posts_df = posts_df.append(preprocess(row[0], row[1], row[2]), ignore_index=True)

with open('hsd/DavidsonEtAl/labeled_data.csv', 'r') as f:
    for d in list(csv.reader(f))[1:]:
        posts_df = posts_df.append({ 'id': '0', 'text': d[6], 'class': d[5]}, ignore_index=True)

posts_df.to_csv('hsd/Reddit/clean_data.csv', index=False)


In [2]:
posts = pd.read_csv('hsd/Reddit/clean_data.csv') 

# with open('hsd/Reddit/perfect_data.pickle', 'wb') as f:
#     for index, row in posts.iterrows():
#         pickle.dump((row['text'], row['class']), f)

In [3]:
posts

,id,text,class
0,39869714,i joined gab to remind myself how retarded jew...,2
1,39845588,This is what the left is really scared of. ...,2
2,39848775,That literally looks like a monkey. Why are we...,2
3,39911017,Dumb Cunt,2
4,37485560,It makes you an asshole.,2
...,...,...,...
79910,0,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1
79911,0,"you've gone and broke the wrong heart baby, an...",2
79912,0,young buck wanna eat!!.. dat nigguh like I ain...,1
79913,0,youu got wild bitches tellin you lies,1
